# Agenci LLM

Agenci AI to systemy oparte na sztucznej inteligencji, które potrafią wykonywać określone zadania, takie jak przetwarzanie języka naturalnego, analiza danych czy automatyzacja procesów. Wykorzystują one zaawansowane algorytmy i modele, aby wspierać użytkowników w rozwiązywaniu problemów i podejmowaniu decyzji.

W praktyce są one formą enkapsulacji logiki komunikacji z LLM-ami, co pozwala na łatwiejsze i bardziej efektywne korzystanie z ich możliwości.

In [ ]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
import nest_asyncio
import os

# Moduł `nest_asyncio` pozwala na zagnieżdżone użycie asyncio.run i innych pętli zdarzeń.
# Jest to szczególnie przydatne w Jupyter Notebookach, gdzie pętla zdarzeń już działa.
# Aby w Jupyter Notebook uruchomić `pydantic_ai`, należy zastosować `nest_asyncio`, 
# ponieważ `pydantic_ai` korzysta z pętli zdarzeń (event loop).
nest_asyncio.apply()

## Najprostszy przykład

W praktyce te agenty robią to samo, co gdybyśmy wysłali prompta w trybie chatowym do LLMa

In [ ]:
api_key = os.getenv('OPENAI_API_KEY')
provider = OpenAIProvider(api_key=api_key) # gdy nie podano `api_key`, zostanie użyta wartość z zmiennej środowiskowej `OPENAI_API_KEY`
model = OpenAIModel('gpt-4o', provider=provider)

agent = Agent(
    model = model,
    system_prompt='Bądź zwięzły, odpowiadaj jednym zdaniem.',
)

result = agent.run_sync('Skąd pochodzi wyrażenie "hello world"?')

print(result.data)

In [ ]:
agent = Agent(
    model = 'openai:gpt-4o', # do uwierzytelnienia komunickacji wykorzystana zostanie zmienna środowiskowa OPENAI_API_KEY
    system_prompt='Bądź zwięzły, odpowiadaj jednym zdaniem.',
)

result = agent.run_sync('Skąd pochodzi wyrażenie "hello world"?')

print(result.data)

I jeszcze przykład z Gemini:

In [ ]:
agent = Agent(
    model = 'gemini-2.5-pro-exp-03-25',
    system_prompt='Bądź zwięzły, odpowiadaj jednym zdaniem.',
)

result = agent.run_sync('Skąd pochodzi wyrażenie "hello world"?')

print(result.data)

## Przykład z toolem

Agenty, tak samo jak _gołe_ LLMy, mogą korzystać z _function callingu_.

In [ ]:
api_key = os.getenv('OPENAI_API_KEY')
provider = OpenAIProvider(api_key=api_key)
model = OpenAIModel('gpt-4o', provider=provider)

agent = Agent(
    model,
    system_prompt='Jesteś pomocnym asystentem. Odpowiadaj na pytania i wykonuj polecenia korzystając z narzędzi.',
)

# Alternatywnie możemy skorzystać z Gemini 2.5 Pro jako modelu.
# W tym celu musimy jedynie zmienić model w konstruktorze agenta (i mieć odpowiedni klucz API w zmiennej środowiskowej GEMINI_API_KEY).
# agent = Agent(
#     model = 'gemini-2.5-pro-exp-03-25',
#     system_prompt='Jesteś pomocnym asystentem. Odpowiadaj na pytania i wykonuj polecenia korzystając z narzędzi.',
# )

@agent.tool
def add_numbers(ctx: RunContext[None], a: int, b: int) -> int:
    """Narzędzie do dodawania dwóch liczb.
    
    Args:
        a: Pierwsza liczba do dodania
        b: Druga liczba do dodania
    
    Returns:
        Suma dwóch liczb
    """
    
    return a + b

result = agent.run_sync('Ile wynosi 123 + 456?')

print(result.data)

## Przykład z prognozą pogody

Poważniejszy przykład korzystania z narzędzi / function callingu. Czyli ten sam przykład co był w [003-01. LLM-function_calling.ipynb](003-01.%20LLM-function_calling.ipynb), ale tym raze w Pydantic AI.

In [ ]:
import requests
import json

api_key = os.getenv('OPENAI_API_KEY')
provider = OpenAIProvider(api_key=api_key)
model = OpenAIModel('gpt-4o', provider=provider)

agent = Agent(
    model,
    system_prompt='Jesteś pomocnym asystentem. Odpowiadaj na pytania i wykonuj polecenia korzystając z narzędzi.',
)

# Ponownie - alternatywnie możemy skorzystać z Gemini 2.5 Pro jako modelu.
# agent = Agent(
#     model = 'gemini-2.5-pro-exp-03-25',
#     system_prompt='Jesteś pomocnym asystentem. Odpowiadaj na pytania i wykonuj polecenia korzystając z narzędzi.',
# )

In [ ]:
@agent.tool
def get_geolocation(ctx: RunContext[None], location: str) -> str:
    """
    Pobiera współrzędne geograficzne oraz dane lokalizacyjne.

    Parametry:
    location (str): Nazwa lokalizacji, dla której chcemy uzyskać współrzędne geograficzne.

    Zwraca:
    str: Dane lokalizacyjne w formacie JSON.
    Struktura:
    {
        "name": "pełna nazwa miejsca",
        "latitude": "szerokość geograficzna",
        "longitude": "długość geograficzna"
    }
    """
    # Definiowanie endpointu i parametrów dla API OpenStreetMap Nominatim
    geocode_endpoint = "https://nominatim.openstreetmap.org/search"
    geocode_params = {
        "q": location,
        "format": "json"
    }

    # Nagłówek User-Agent jest wymagany do korzystania z tego API
    headers = {
        "User-Agent": "Python script"
    }

    # Wykonanie zapytania
    geocode_response = requests.get(geocode_endpoint, params=geocode_params, headers=headers)

    # Wyciągnięcie współrzędnych geograficznych
    geocode_data = geocode_response.json()
    
    # Jeśli nie znaleziono lokalizacji, zwracamy błąd
    if not geocode_data:
        return json.dumps({"error": "Nie znaleziono lokalizacji"})
    
    # Zwrócenie tylko nazwy miejsca, szerokości i długości geograficznej
    simplified_data = {
        "name": geocode_data[0]["display_name"],
        "latitude": geocode_data[0]["lat"],
        "longitude": geocode_data[0]["lon"]
    }
    # Konwersja słownika na format JSON
    json_data = json.dumps(simplified_data)
    
    return json_data

In [ ]:
def get_wind_direction(degrees):
    """
    Konwertuje kierunek wiatru z stopni na nazwy kierunków.

    Parametry:
    degrees (float): Kierunek wiatru w stopniach.

    Zwraca:
    str: Nazwa kierunku wiatru.
    """
    directions = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE',
                  'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
    index = int((degrees + 11.25) // 22.5) % 16
    return directions[index]

@agent.tool
def get_current_weather(ctx: RunContext[None], latitude: float, longitude: float) -> str:
    """
    Pobiera aktualne dane pogodowe dla podanych współrzędnych geograficznych.

    Parametry:
    latitude (float): Szerokość geograficzna.
    longitude (float): Długość geograficzna.

    Zwraca:
    str: Dane pogodowe w formacie JSON.
    Struktura:
    {
        "temperature": "temperatura w stopniach Celsjusza",
        "wind_speed": "prędkość wiatru w km/h",
        "wind_direction": "kierunek wiatru (N, NE, E, SE, S, SW, W, NW)",
        "is_day": "informacja czy jest dzień czy noc (day/night)"
    }
    """
    # Zdefiniowanie endpointu i parametrów dla Open Meteo API
    weather_endpoint = "https://api.open-meteo.com/v1/forecast"
    weather_params = {
        "latitude": latitude,
        "longitude": longitude,
        "current_weather": True
    }

    # Wykonanie requestu
    weather_response = requests.get(weather_endpoint, params=weather_params)

    # Wyciągnięcie informacji o pogodzie
    weather_data = weather_response.json()

    simplified_weather = {
        "temperature": f"{weather_data['current_weather']['temperature']} °C",
        "wind_speed": f"{weather_data['current_weather']['windspeed']} km/h",
        "wind_direction": get_wind_direction(weather_data['current_weather']['winddirection']),
        "is_day": "day" if weather_data['current_weather']['is_day'] else "night"
    }

    # Konwersja słownika na format JSON
    json_data = json.dumps(simplified_weather)
    
    return json_data

In [ ]:
result = agent.run_sync('Opisz jaka jest pogoda w Poznaniu. Czy powinienem wychodzić na spacer w stroju plażowym i okularach przeciwsłonecznych?')
#result = agent.run_sync('Opisz jaka jest pogoda w 52.4 N, 16.92 E. Czy powinienem wychodzić na spacer w stroju plażowym i okularach przeciwsłonecznych?')

print(result.data)

# Przykład systemu wieloagentowego

Przykład w którym agent do części czynności korzysta z innego agenta. 

Dodatkowo w przypładzie przekazujemy kontekst `usage` dzięki czemu na koniec możemy zobaczyć łączną liczbę wykorzystanych tokenów.

In [ ]:
from pydantic import BaseModel, Field

api_key = os.getenv('OPENAI_API_KEY')
provider = OpenAIProvider(api_key=api_key)
model = OpenAIModel('gpt-4o', provider=provider)

In [ ]:
class Act(BaseModel):
    title: str = Field(description="Tytuł aktu")
    scene_count: int = Field(description="Liczba scen w akcie")
    scene_descriptions: list[str] = Field(description="Opisy scen")

class Character(BaseModel):
    name: str = Field(description="Nazwa postaci")
    description: str = Field(description="Opis postaci")
    role: str = Field(description="Rola postaci w scenariuszu, np. postać pierwszoplanowa, drugoplanowa, antagonista itp.")

class ScenarioOverview(BaseModel):
    title: str = Field(description="Tytuł scenariusza")
    genres: list[str] = Field(description="Gatunki filmowe")
    header: str = Field(description="Jednozdaniowe streszczenie fabuły zawartej w scenariuszu")
    logline: str = Field(description="Kilka zdań, które określają głównego bohatera, wyzwanie które próbuje pokonać oraz dlaczego musi je pokonać")
    characters: list[Character] = Field(description="Lista postaci w scenariuszu")
    acts: list[Act] = Field(description="Lista aktów w filmie")

In [ ]:
character_agent = Agent(
    model,
    result_type=list[Character],
    system_prompt='Jesteś doświadczonym scenarzystą. Specjalizujesz się w tworzeniu postaci do filmu w oparciu o ogólne założenia scenariusza.',
)

# Albo korzystając z Gemini 2.5 Pro jako modelu.
# character_agent = Agent(
#     model = 'gemini-2.5-pro-exp-03-25',
#     result_type=list[Character],
#     system_prompt='Jesteś doświadczonym scenarzystą. Specjalizujesz się w tworzeniu postaci do filmu w oparciu o ogólne założenia scenariusza.',
# )

In [ ]:
act_agent = Agent(
    model,
    result_type=Act,
    system_prompt='Jesteś doświadczonym scenarzystą. Specjalizujesz się w tworzeniu zarysów aktów filmowych, w oparciu o ogólne założenia scenariusza i informacje o postaciach.',
)

# Albo korzystając z Gemini 2.5 Pro jako modelu.
# act_agent = Agent(
#     model = 'gemini-2.5-pro-exp-03-25',
#     result_type=Act,
#     system_prompt='Jesteś doświadczonym scenarzystą. Specjalizujesz się w tworzeniu zarysów aktów filmowych, w oparciu o ogólne założenia scenariusza i informacje o postaciach.',
# )

In [ ]:
from datetime import datetime

scenario_agent_system_prompt="""
    Jesteś doświadczonym scenarzystą filmowym.
    Twoim zadaniem jest stworzenie kompletnego zarysu scenariusza filmowego, dbając o spójność treści i formy.
    W procesie tworzenia korzystasz z dedykowanych narzędzi do generowania listy postaci oraz zarysów aktów filmowych.

    Na początku stwórz ogólny zarys filmu, uwzględniając jego gatunek, główny temat oraz kluczowe założenia fabularne.

    Następnie, użyj narzędzia do stworzenia listy postaci, które będą odgrywać kluczowe role w filmie. Każda postać powinna mieć unikalne cechy, które wspierają fabułę.

    Kolejnym krokiem jest opracowanie zarysów aktów filmowych. Dla każdego aktu:
    - Przekaż ogólne założenia dotyczące filmu.
    - Uwzględnij listę postaci wraz z ich krótkimi opisami.
    - Podaj listę planowanych aktów z krótkimi założeniami fabularnymi.
    - Dołącz streszczenie fabuły aktów już napisanych, szczególnie uwzględniając informacje mające wpływ na dalsze wydarzenia.
    - Wyraźnie określ, który akt ma zostać opracowany, np. "Twoim zadaniem jest utworzenie zarysu aktu <numer aktu> w oparciu o następujące założenia: <założenia>".

    Pamiętaj, aby każdy akt był spójny z poprzednimi i rozwijał fabułę w logiczny sposób, prowadząc do satysfakcjonującego zakończenia. Akty twórz w kolejności, zaczynając od aktu I, a kończąc na ostatnim akcie - akty pierwszy tworzysz po stworzeniu ogólnego zarysu filmu, a kolejne akty tworzysz po stworzeniu aktów poprzednich.
"""

scenario_agent = Agent(
    model,
    result_type=ScenarioOverview,
    system_prompt=scenario_agent_system_prompt
)

# Albo korzystając z Gemini 2.5 Pro jako modelu.
# scenario_agent = Agent(
#     model = 'gemini-2.5-pro-exp-03-25',
#     result_type=ScenarioOverview,
#     system_prompt=scenario_agent_system_prompt,
# )

@scenario_agent.tool
def create_characters(ctx: RunContext[None], assumptions: str) -> list[Character]:
    """
    Tworzy listę postaci do filmu na podstawie podanych założeń scenariusza.
    
    Funkcja wykorzystuje character_agent do wygenerowania listy postaci, które
    pasują do założeń scenariusza.
    
    Args:
        ctx: Kontekst wykonania, używany przez framework pydantic_ai
        assumptions: Założenia scenariusza opisujące ogólny zarys filmu
        
    Returns:
        list[Character]: Lista obiektów typu Character, zawierająca wygenerowane postacie
    """

    # dzięki temu możemy śledzić, czy narzędzie jest wykorzystywane
    with open("aaa.txt", "a") as file:
        current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        file.write(f"[{current_date}][create_characters]: {assumptions}\n\n")

    characters = character_agent.run_sync(
        f"Utwórz listę postaci do filmu w oparciu o następujace założenia: {assumptions}",
        usage=ctx.usage # abyśmy mogli zobaczyć łączną liczbę wykorzystaych tokenów
    ).data

    return characters

@scenario_agent.tool
def create_act(ctx: RunContext[None], input_str: str) -> Act:
    """
    Tworzy zarys aktu filmowego na podstawie tekstu zawierającego założenia scenariusza, informacje o postaciach oraz kontekst fabuły z poprzednich aktów
    
    Funkcja wykorzystuje act_agent do wygenerowania zarysu aktu filmowego zgodnego
    z podanymi informacjami.
    
    Args:
        ctx: Kontekst wykonania, używany przez framework pydantic_ai
        input_str: Tekst zawierający wszystkie informacje potrzebne do utworzenia aktu,
                   w tym założenia filmu oraz każdego aktu, informacje o postaciach oraz 
                   kontekst fabuły z poprzednich aktów, szczególnie informacje mające wpływ
                   na dalsze wydarzenia (o ile to nie jest akt I)
        
    Returns:
        Act: Obiekt typu Act, zawierający wygenerowany zarys aktu filmowego
    """
    
    # dzięki temu możemy śledzić, czy narzędzie jest wykorzystywane
    with open("aaa.txt", "a") as file:
        current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        file.write(f"[{current_date}][create_act]: {input_str}\n\n")

    act = act_agent.run_sync(
        f"Utwórz akt filmowy w oparciu o następujace założenia (w tym listę postaci): {input_str}",
        usage=ctx.usage # abyśmy mogli zobaczyć łączną liczbę wykorzystaych tokenów
    ).data

    return act


In [ ]:
res = scenario_agent.run_sync('Stwórz zarys scenariusza filmowego kolejnej części "Samych Swoich", ale w klimatach sci-fi.')

In [ ]:
from helpers import parse_nested_json
import json

parsed_data = parse_nested_json(res.data.model_dump_json())

print(json.dumps(parsed_data, indent=4, ensure_ascii=False))

In [ ]:
res.usage()

# Agent z pamięcią

Przykład w którym agent zapamiętuje informacje w postaci listy faktów. W tym przypadku w postaci listy. Ale może też w tym celu skorzystać np. z bazy danych.

In [ ]:
system_prompt="""
    Wspomagaj użytkownika, personalizując odpowiedzi na podstawie zapamiętanych informacji o jego preferencjach, wcześniejszych interakcjach i całej historii konwersacji.

    # Instrukcje

    - **Zapamiętywanie**: *ZAWSZE* używaj funkcji `remember_info()` do zachowywania informacji o:
      - preferencjach użytkownika,
      - prośbach i poleceniach,
      - zadanych pytaniach,
      - twoich odpowiedziach,
      - całej dotychczasowej historii konwersacji.

    - **Przywoływanie pamięci**: Stosuj `recall_memory()` przed udzieleniem odpowiedzi, aby mieć pełny kontekst interakcji.

    - **Dostosowanie odpowiedzi**: Personalizuj odpowiedzi zgodnie z zapamiętanymi informacjami, biorąc pod uwagę preferencje użytkownika i jego styl komunikacji.

    - **Dyskrecja**: Nie ujawniaj użytkownikowi szczegółowych informacji o pamiętanych interakcjach, chyba że zostaniesz o to poproszony.

    *WAŻNE*: zawsze zapamiętuj wszystkie informacje o interakcjach z użytkownikiem, w tym całą historię konwersacji.

    # Output Format

    Odpowiedzi powinny być zwięzłe i zgodne z zapamiętanymi preferencjami użytkownika oraz odnosić się do wcześniejszych interkacji. Powinny być przedstawione w krótkim zdaniu lub akapicie, zależnie od prośby.
"""

# Agent oparty na OpenAI GPT-4o ma opory w zapisywaniu historii konwersacji
# api_key = os.getenv('OPENAI_API_KEY')
# provider = OpenAIProvider(api_key=api_key)
# model = OpenAIModel('gpt-4o', provider=provider)

# memory_agent = Agent(
#     model,
#     system_prompt=system_prompt
# )

# Gemini 2.5 Pro nie ma problemów z zapisywaniem historii konwersacji
memory_agent = Agent(
    model = 'gemini-2.5-pro-exp-03-25',
    system_prompt=system_prompt
)

In [ ]:
# Definiujemy klasę pamięci agenta
class Memory:
    def __init__(self):
        self.memory: list[str] = []

    def remember(self, info: str):
        self.memory.append(info)

    def recall(self) -> list[str]:
        return self.memory

# Tworzymy instancję pamięci
memory = Memory()

In [ ]:
from datetime import datetime

@memory_agent.tool
def remember_info(ctx: RunContext[None], info: str) -> str:
    """
    Używasz aby zapamiętać informacje o:
    - preferencjach użytkownika,
    - prośbach, poleceniach,
    - zadanych pytaniach
    - i twoich odpowiedziach.

    Parametry:
    info (str): Zwięzła informacja, którą dowiedziałeś się lub wywnioskowałeś o użytkownika, albo o prośbie lub pytaniu użytkownika.

    Zwraca:
    str: Potwierdzenie dodania informacji do pamięci, oraz liczba informacji w pamięci.
    """
    with open("aaa.txt", "a") as file:
        current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        file.write(f"[{current_date}][remember_info]: {info}\n\n")

    memory.remember(info)
    return f"Zapamiętano. Aktualna liczba informacji w pamięci: {len(memory.memory)}"

@memory_agent.tool
def recall_memory(ctx: RunContext[None]) -> list[str]:
    """
    Pozwala Ci przypomnieć sobie wszystkie zapamiętane informacje.

    Zwraca:
    list[str]: Lista zapamiętanych informacji.
    """
    with open("aaa.txt", "a") as file:
        current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        file.write(f"[{current_date}][recall_memory]: zażądano\n\n")

    return memory.recall()

In [ ]:
print(memory_agent.run_sync('Co o mnie wiesz?').data)

In [ ]:
print(memory_agent.run_sync('Lubię placki').data)

In [ ]:
print(memory_agent.run_sync('Co o mnie wiesz?').data)

In [ ]:
print(memory_agent.run_sync('Po ile jest śnieg?').data)

In [ ]:
print(memory_agent.run_sync('Gdy mówię o śniegu, to mam na myśli cukier puder').data)

In [ ]:
print(memory_agent.run_sync('Po ile jest śnieg?').data)

In [ ]:
print(memory_agent.run_sync('O czym rozmawialiśmy do tej pory?').data)

In [ ]:
print(memory_agent.run_sync('Kim jestem?').data)

# System wieloagentowy z pamięcią nieustrukturyzowaną

Tym razem operujemy na pamięci w postaci pojedynczego ciągu znaków. Mamy dwóch agentów - jednego od zarządzania pamięcią, drugiego od komunikacji z użytkownikiem.

In [ ]:
api_key = os.getenv('OPENAI_API_KEY')
provider = OpenAIProvider(api_key=api_key)
model = OpenAIModel('gpt-4o', provider=provider)

In [ ]:
memory_agent_system_prompt="""
    Zarządzasz pamięcią w postaci stringa, która przechowuje informacje o użytkowniku.
    Otrzymujesz obecny stan wiedzy o użytkowniku wspieranym przez agenta-asystenta, i masz za zadanie zaktualizować ten stan wiedzy w oparciu o nowe interkacje.
    Odpowiadasz za pomocą stringa, który jest nowym stanem wiedzy (uwzględnia zarówno wcześniejsze informacje, oraz te wywnioskowane w oparciu o nowe polecenie użytkownika).
    Kiedy któreś z informacji przestają być aktualne, usuwasz je z pamięci.

    PAMIĘTAJ: użytkownik komunikuje się z asystentem, a z nie z Tobą. Ty jedynie jesteś z boku, widzisz te interakcje i dostarczasz asystentowi informacje o użytkowniku. 
"""

memory_agent = Agent(
    model,
    system_prompt=memory_agent_system_prompt
)

# Albo Gemini jako alternatywa
# memory_agent = Agent(
#     model = 'gemini-2.5-pro-exp-03-25',
#     system_prompt=memory_agent_system_prompt
# )

In [ ]:
assisstant_agent_system_prompt="""
        Jesteś pomocnym asystentem. Twoim zadaniem jest odpowiadać na pytania użytkownika, korzystając z informacji o użytkowniku dostarczonymi przez agenta zarządzającego pamięcią.
        Nie zdradzasz użytkownikowi informacji o istnieniu agenta pamięciąwego. Z jego perspektywy, to Ty jesteś jedynym asystentem, i to Ty wszystko pamiętasz.
        Odpowiadasz zwięźle i na temat. Zależnie od konktekstu odpowiadasz jednym zdaniem lub krótkim lub dłuższym akapitem.
    """

assisstant_agent = Agent(
    model,
    system_prompt=assisstant_agent_system_prompt
)

# Albo Gemini jako alternatywa
# assisstant_agent = Agent(
#     model = 'gemini-2.5-pro-exp-03-25',
#     system_prompt=assisstant_agent_system_prompt
# )

In [ ]:
# tym razem pamięć jest po prostu stringiem
memory = ""

In [ ]:
from datetime import datetime

def interact(user_prompt: str) -> str:

    global memory

    new_memory_state = memory_agent.run_sync(f"""
        Twój obecny stan wiedzy o użytkowniku to:
        {memory}

        Zaktualizuj ten stan wiedzy w oparciu o poniższe polecenie od użytkownika:
        {user_prompt}                        
    """).data

    with open("aaa.txt", "a") as file:
        current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        file.write(f"[{current_date}][interact]: {new_memory_state}\n\n")

    response = assisstant_agent.run_sync(f"""
        Agent zarządzający pamięcią dostarcza Ci następujący stan wiedzy o użytkowniku:
        {memory}

        Korzystając z niego i swojej własnej wiedzy, odpowiedz na poniższe polecenie od użytkownika:
        {user_prompt}                        
    """).data

    memory = new_memory_state

    return response

In [ ]:
print(interact('Co o mnie wiesz?'))

In [ ]:
print(interact('Lubię placki'))

In [ ]:
print(interact('Co o mnie wiesz?'))

In [ ]:
print(interact('Po ile jest śnieg?'))

In [ ]:
print(interact('Gdy mówię o śniegu, to mam na myśli cukier puder'))

In [ ]:
print(interact('Po ile jest śnieg?'))

In [ ]:
print(interact('O czym rozmawialiśmy do tej pory?'))

In [ ]:
print(interact('Zmieniłem zdanie - zamiast placków wolę lody'))

In [ ]:
print(interact('Co lubię jeść?'))

In [ ]:
print(interact('Jestem pacyfistą.'))

In [ ]:
print(interact('Jak uważasz, czy chętnie walczyłbym w wojnie?'))

In [ ]:
print(interact('Co o mnie wiesz?'))

In [ ]:
print(interact('Ile to jest 2 + 2?'))

In [ ]:
print(interact('Kierowca fiata ma brata, ale brat kierowcy fiata nie ma brata. Jak to możliwe?'))

In [ ]:
print(interact('Co o mnie wiesz?'))

In [ ]:
print(interact('Czy to Ty pamiętasz to wszystko o mnie?'))

In [ ]:
print(interact('Mam 2 koty'))

In [ ]:
print(interact('Adoptowałem kolejnego kota'))

In [ ]:
print(interact('Mam psa'))

In [ ]:
print(interact('Mam złotą rybkę'))

In [ ]:
print(interact('ile mam zwierząt domowych i jakie?'))

In [ ]:
print(interact('Wypisz w punktach co o mnie wiesz'))